# Homework 10: `SQL`

## Due Date:  Thursday, November 16th at 11:59 PM

You will create a database of the NASA polynomial coefficients for each specie.

**Please turn in your database with your `Jupyter` notebook!**

# Question 1: Convert XML to a SQL database

Create two tables named `LOW` and `HIGH`, each corresponding to data given for the low and high temperature range.
Each should have the following column names:

- `SPECIES_NAME`
- `TLOW`
- `THIGH`
- `COEFF_1`
- `COEFF_2`
- `COEFF_3`
- `COEFF_4`
- `COEFF_5`
- `COEFF_6`
- `COEFF_7`

Populate the tables using the XML file you created in last assignment. If you did not complete the last assignment, you may also use the `example_thermo.xml` file.

`TLOW` should refer to the temperature at the low range and `THIGH` should refer to the temperature at the high range.  For example, in the `LOW` table, $H$ would have `TLOW` at $200$ and `THIGH` at $1000$ and in the `HIGH` table, $H$ would have `TLOW` at $1000$ and `THIGH` at $3500$.

For both tables, `COEFF_1` through `COEFF_7` should be populated with the corresponding coefficients for the low temperature data and high temperature data.

In [1]:
import xml.etree.ElementTree as ET
import sqlite3
import pandas as pd
from IPython.core.display import display

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

# Create and connect to database
db = sqlite3.connect('NASA_coef.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS LOW")
cursor.execute("DROP TABLE IF EXISTS HIGH")
cursor.execute("PRAGMA foreign_keys=1")

# Create the table for low temperature range
cursor.execute('''CREATE TABLE LOW (
               id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
               SPECIES_NAME TEXT,
               TLOW REAL, 
               THIGH REAL, 
               COEFF_1 REAL, 
               COEFF_2 REAL,
               COEFF_3 REAL,
               COEFF_4 REAL,
               COEFF_5 REAL,
               COEFF_6 REAL,
               COEFF_7 REAL)''')

db.commit()

# Create the table for high temperature range
cursor.execute('''CREATE TABLE HIGH (
               id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
               SPECIES_NAME TEXT,
               TLOW REAL, 
               THIGH REAL, 
               COEFF_1 REAL, 
               COEFF_2 REAL,
               COEFF_3 REAL,
               COEFF_4 REAL,
               COEFF_5 REAL,
               COEFF_6 REAL,
               COEFF_7 REAL)''')
db.commit()

# The given helper function (from L18) to visualize tables
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = []
    for i, col_name in enumerate(cols):
        framelist.append((col_name, [col[i] for col in q]))
    return pd.DataFrame.from_items(framelist)



In [2]:
tree = ET.parse('thermo.xml')

species_data = tree.find('speciesData')
species_list = species_data.findall('species') # a list of all species

for species in species_list:
    name = species.get('name')
    NASA_list = species.find('thermo').findall('NASA')
    for NASA in NASA_list:
        T_min = float(NASA.get('Tmin'))
        T_max = float(NASA.get('Tmax'))
        coefs = NASA.find('floatArray').text.split(',')
        vals_to_insert = (name, T_min, T_max, float(coefs[0].strip()), float(coefs[1].strip()), 
                          float(coefs[2].strip()), float(coefs[3].strip()), float(coefs[4].strip()), 
                          float(coefs[5].strip()), float(coefs[6].strip())) 

        if T_max > 1000: # high range temperature
            cursor.execute('''INSERT INTO HIGH 
                  (SPECIES_NAME, TLOW, THIGH, COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7)
                  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)
        
        else: # low range temperature
            cursor.execute('''INSERT INTO LOW 
                  (SPECIES_NAME, TLOW, THIGH, COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7)
                  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

In [3]:
LOW_cols = [col[1] for col in cursor.execute("PRAGMA table_info(LOW)")]
display(viz_tables(LOW_cols, '''SELECT * FROM LOW'''))

,id,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,1,O,200.0,1000.0,3.168267,-3.279319e-03,6.643064e-06,-6.128066e-09,2.112660e-12,29122.259200,2.051933
1,2,O2,200.0,1000.0,3.782456,-2.996734e-03,9.847302e-06,-9.681295e-09,3.243728e-12,-1063.943560,3.657676
2,3,H,200.0,1000.0,2.500000,7.053328e-13,-1.995920e-15,2.300816e-18,-9.277323e-22,25473.659900,-0.446683
3,4,H2,200.0,1000.0,2.344331,7.980521e-03,-1.947815e-05,2.015721e-08,-7.376118e-12,-917.935173,0.683010
4,5,OH,200.0,1000.0,3.992015,-2.401318e-03,4.617938e-06,-3.881133e-09,1.364115e-12,3615.080560,-0.103925
5,6,H2O,200.0,1000.0,4.198641,-2.036434e-03,6.520402e-06,-5.487971e-09,1.771978e-12,-30293.726700,-0.849032
6,7,HO2,200.0,1000.0,4.301798,-4.749121e-03,2.115829e-05,-2.427639e-08,9.292251e-12,294.808040,3.716662
7,8,H2O2,200.0,1000.0,4.276113,-5.428224e-04,1.673357e-05,-2.157708e-08,8.624544e-12,-17702.582100,3.435051


In [4]:
HIGH_cols = [col[1] for col in cursor.execute("PRAGMA table_info(HIGH)")]
display(viz_tables(HIGH_cols, '''SELECT * FROM HIGH'''))

,id,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,1,O,1000.0,3500.0,2.569421,-8.597411e-05,4.194846e-08,-1.001778e-11,1.228337e-15,29217.579100,4.784339
1,2,O2,1000.0,3500.0,3.282538,1.483088e-03,-7.579667e-07,2.094706e-10,-2.167178e-14,-1088.457720,5.453231
2,3,H,1000.0,3500.0,2.500000,-2.308430e-11,1.615619e-14,-4.735152e-18,4.981974e-22,25473.659900,-0.446683
3,4,H2,1000.0,3500.0,3.337279,-4.940247e-05,4.994568e-07,-1.795664e-10,2.002554e-14,-950.158922,-3.205023
4,5,OH,1000.0,3500.0,3.092888,5.484297e-04,1.265052e-07,-8.794616e-11,1.174124e-14,3858.657000,4.476696
5,6,H2O,1000.0,3500.0,3.033992,2.176918e-03,-1.640725e-07,-9.704199e-11,1.682010e-14,-30004.297100,4.966770
6,7,HO2,1000.0,3500.0,4.017211,2.239820e-03,-6.336581e-07,1.142464e-10,-1.079085e-14,111.856713,3.785102
7,8,H2O2,1000.0,3500.0,4.165003,4.908317e-03,-1.901392e-06,3.711860e-10,-2.879083e-14,-17861.787700,2.916157


# Question 2: `WHERE` Statements

1. Write a `Python` function `get_coeffs` that returns an array of 7 coefficients.  
   
   The function should take in two parameters: 1.) `species_name` and 2.) `temp_range`, an indicator variable ('low' or 'high') to indicate whether the coefficients should come from the low or high temperature range.  
   The function should use `SQL` commands and `WHERE` statements on the table you just created in Question 1 (rather than taking data from the XML directly).
```python
def get_coeffs(species_name, temp_range):
    ''' Fill in here'''
    return coeffs
```

2. Write a python function `get_species` that returns all species that have a temperature range above or below a given value. The function should take in two parameters: 1.) `temp` and 2.) `temp_range`, an indicator variable ('low' or 'high').

  When temp_range is 'low', we are looking for species with a temperature range lower than the given temperature, and for a 'high' temp_range, we want species with a temperature range higher than the given temperature.

  This exercise may be useful if different species have different `LOW` and `HIGH` ranges.

  And as before, you should accomplish this through `SQL` queries and where statements.

```python
def get_species(temp, temp_range):
    ''' Fill in here'''
    return coeffs
```

In [5]:
def get_coeffs(species_name, temp_range):
    query = '''SELECT COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7
                FROM {} 
                WHERE SPECIES_NAME = "{}"'''.format(temp_range.upper(), species_name)
    coeffs = list(cursor.execute(query).fetchall()[0])
    return coeffs

In [6]:
get_coeffs('H', 'low')

[2.5,
 7.05332819e-13,
 -1.99591964e-15,
 2.30081632e-18,
 -9.27732332e-22,
 25473.6599,
 -0.446682853]

In [7]:
def get_species(temp, temp_range):
    if temp_range == 'low': # temp_range == 'low'
        query = '''SELECT SPECIES_NAME FROM {} WHERE TLOW < {}'''.format(temp_range.upper(), temp)
    else: # temp_range == 'high'
        query = '''SELECT SPECIES_NAME FROM {} WHERE THIGH > {}'''.format(temp_range.upper(), temp)
    species = []
    for s in cursor.execute(query).fetchall():
        species.append(s[0])
    return species

In [8]:
get_species(500, 'low')

['O', 'O2', 'H', 'H2', 'OH', 'H2O', 'HO2', 'H2O2']

In [9]:
get_species(100, 'low')

[]

In [10]:
get_species(3000, 'high')

['O', 'O2', 'H', 'H2', 'OH', 'H2O', 'HO2', 'H2O2']

In [11]:
get_species(3500, 'high')

[]

# Question 3: `JOIN` STATEMENTS

Create a table named `ALL_TEMPS` that has the following columns:

- `SPECIES_NAME`
- `TEMP_LOW`
- `TEMP_HIGH`

This table should be created by joining the tables `LOW` and `HIGH` on the value `SPECIES_NAME`.

In [12]:
# Create the table for ALL_TEMPS
cursor.execute("DROP TABLE IF EXISTS ALL_TEMPS")

cursor.execute('''CREATE TABLE ALL_TEMPS (
               id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
               SPECIES_NAME TEXT,
               TEMP_LOW REAL, 
               TEMP_HIGH REAL)''')
db.commit()

# Insert TEMP_LOW and TEMP_HIGH of all species to ALL_TEMPS
query = '''SELECT LOW.SPECIES_NAME, LOW.TLOW AS TEMP_LOW, HIGH.THIGH AS TEMP_HIGH
        FROM LOW
        INNER JOIN HIGH ON LOW.SPECIES_NAME = HIGH.SPECIES_NAME'''

for record in cursor.execute(query).fetchall():
    cursor.execute('''INSERT INTO ALL_TEMPS 
                  (SPECIES_NAME, TEMP_LOW, TEMP_HIGH)
                  VALUES (?, ?, ?)''', record)

In [13]:
ALL_TEMPS_cols = [col[1] for col in cursor.execute("PRAGMA table_info(ALL_TEMPS)")]
display(viz_tables(ALL_TEMPS_cols, '''SELECT * FROM ALL_TEMPS'''))

,id,SPECIES_NAME,TEMP_LOW,TEMP_HIGH
0,1,O,200.0,3500.0
1,2,O2,200.0,3500.0
2,3,H,200.0,3500.0
3,4,H2,200.0,3500.0
4,5,OH,200.0,3500.0
5,6,H2O,200.0,3500.0
6,7,HO2,200.0,3500.0
7,8,H2O2,200.0,3500.0


1. Write a `Python` function `get_range` that returns the range of temperatures for a given species_name.

The range should be computed within the `SQL` query (i.e. you should subtract within the `SELECT` statement in the `SQL` query).
```python
def get_range(species_name):
    '''Fill in here'''
    return range
```

Note that `TEMP_LOW` is the lowest temperature in the `LOW` range and `TEMP_HIGH` is the highest temperature in the `HIGH` range.

In [14]:
def get_range(species_name):
    query = '''SELECT (TEMP_HIGH - TEMP_LOW) AS T_range FROM ALL_TEMPS WHERE SPECIES_NAME = "{}"'''.format(species_name)
    T_range = cursor.execute(query).fetchall()[0][0]
    return T_range

In [15]:
get_range('O')

3300.0

In [16]:
# Close the Database
db.commit()
db.close()